In [1]:
%%capture

!apt-get update
!apt-get install -y openjdk-8-jdk-headless -qq
!apt-get install maven -qq

!curl -L "https://archive.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz" > spark-2.4.5-bin-hadoop2.7.tgz
!tar -xvf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install pyspark py4j
from pyspark.sql import SparkSession
from pyspark.sql.functions import when
from pyspark.sql import functions
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col
#from pyspark.sql.Column import isNull
from pyspark.sql.functions import lit
spark = SparkSession.builder.master("local[*]").config("spark.driver.memory", "16g").getOrCreate()

In [41]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,lit,explode
import pyspark.sql.functions as f   # new thing

from google.colab import files

In [57]:
# EXTRACTING
#uploaded = files.upload()
driver_path = "JAR/postgresql-42.7.1.jar"

spark=SparkSession.builder.appName("ETL Pipeline").config("spark.jars",driver_path).getOrCreate()
df=spark.read.text("WordData.txt")
df.show()


+--------------------+
|               value|
+--------------------+
|This is a Japanes...|
|The team members ...|
|As the years pass...|
|If you don't like...|
|He was disappoint...|
|When he encounter...|
|Situps are a terr...|
|Toddlers feeding ...|
|Edith could decid...|
|Her daily goal wa...|
|Tomorrow will bri...|
|His son quipped t...|
|He wondered why a...|
|If my calculator ...|
|The hummingbird's...|
|He went on a whis...|
|This is the last ...|
|I come from a tri...|
|The delicious aro...|
|Weather is not tr...|
+--------------------+
only showing top 20 rows



In [58]:
# TRANSFRMING


df2=df.withColumn("splitted_data",f.split("value"," "))  # splitting the value column thrugh f function
df3=df2.withColumn("words",explode("splitted_data"))  # exploading the array/list into single
df3.show()

wordsDf=df3.select(col("words"))
wordsDf.show()

wordCount = wordsDf.groupBy("words").count()





+--------------------+--------------------+--------+
|               value|       splitted_data|   words|
+--------------------+--------------------+--------+
|This is a Japanes...|[This, is, a, Jap...|    This|
|This is a Japanes...|[This, is, a, Jap...|      is|
|This is a Japanes...|[This, is, a, Jap...|       a|
|This is a Japanes...|[This, is, a, Jap...|Japanese|
|This is a Japanes...|[This, is, a, Jap...|    doll|
|The team members ...|[The, team, membe...|     The|
|The team members ...|[The, team, membe...|    team|
|The team members ...|[The, team, membe...| members|
|The team members ...|[The, team, membe...|    were|
|The team members ...|[The, team, membe...|    hard|
|The team members ...|[The, team, membe...|      to|
|The team members ...|[The, team, membe...|    tell|
|The team members ...|[The, team, membe...|   apart|
|The team members ...|[The, team, membe...|   since|
|The team members ...|[The, team, membe...|    they|
|The team members ...|[The, team, membe...|   

In [60]:
# we have to load connection to RDS.


driver="org.postgresql.Driver"
url="jdbc:postgresql://database-1.c7c26smkstx4.us-east-1.rds.amazonaws.com/postgres"
table="nishant_schema_pyspark.WordCount"
user="postgres"
password="postgres"


wordCount.write.format("jdbc") \
.option("driver", driver).option("url",url).option("dbtable", table) \
.option("user",user).option("password", password).option("mode"
,"append").save()




Py4JJavaError: An error occurred while calling o561.save.
: java.lang.ClassNotFoundException: org.postgresql.Driver
	at java.base/java.net.URLClassLoader.findClass(URLClassLoader.java:476)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:594)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:527)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverRegistry$.register(DriverRegistry.scala:46)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1(JDBCOptions.scala:103)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1$adapted(JDBCOptions.scala:103)
	at scala.Option.foreach(Option.scala:407)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:103)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:246)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:250)
	at or